In [1]:
# importing required libraries

import sys
#sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf

from reco_utils.recommender.ncf.ncf_singlenode import NCF


from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

2021-08-27 15:30:50.714445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-27 15:30:50.714504: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.common.constants import SEED as DEFAULT_SEED

In [3]:

# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

# loading movie leans data
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE,header=["userID", "itemID", "rating", "timestamp"])


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4.81k/4.81k [00:05<00:00, 941KB/s]


In [4]:
df.head()

,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [5]:
# chronological spilt on every user
train, test = python_chrono_split(df, 0.75)

# preparing the data
# This function by default takes sampling on unobserved entries to account for negative feedback 
# It only takes rows which atleast have user id, item id and rating
# I know we discussed the benifits of using implicit data but NCF works for explicit data too 
# This data contains rating and hence is the textbook definition of explicit data
data = NCFDataset(train=train, test=test, seed=SEED)


In [9]:
# to solve the error of placeholder --->tf.compat.v1.disable_eager_execution()
model = NCF(n_users=data.n_users,n_items=data.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)


RuntimeError: tf.placeholder() is not compatible with eager execution.

In [ ]:
# n_factors (int): Dimension of latent space.
# layer_sizes (list): Number of layers for MLP.

# training the model
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# saving the predictions in a dataframe
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()